In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader, random_split, Subset
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt
import torch.optim as optim
import torch.nn.functional as F



In [5]:
# Define specified CNN  architecture for CIFAR-100
class CNNCifar(nn.Module):
  def __init__(self, num_classes = 100):
    super(CNNCifar, self).__init__()
    self.conv1 = nn.Conv2d(3,64,kernel_size=5, padding=2)
    self.pool1 = nn.MaxPool2d(2,2)
    self.conv2 = nn.Conv2d(64,64, kernel_size= 5, padding=2)
    self.pool2 = nn.MaxPool2d(2,2)
    self.fc1 = nn.Linear(64 * 8 * 8, 384)
    self.fc2 = nn.Linear(384, 192)
    self.classifier = nn.Linear(192, num_classes)

  def forward(self, x):
    x = self.conv1(x)
    x = self.pool1(x)
    x = self.conv2(x)
    x = self.pool2(x)
    x = x.view(-1, 64 * 8 * 8)
    x = self.fc1(x)
    x = self.fc2(x)
    x = self.classifier(x)
    return x # the predicted class scores for the input image batch



NameError: name 'nn' is not defined

In [3]:
from torchvision import transforms

transform_train= transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4865, 0.4409), (0.2673, 0.2564, 0.2761))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4865, 0.4409), (0.2673, 0.2564, 0.2761))
])


In [4]:
trainset = datasets.CIFAR100(root='./data', train=True, download=True, transform=transform_train)
testset = datasets.CIFAR100(root='./data', train=False, download=True, transform=transform_test)


100%|██████████| 169M/169M [00:05<00:00, 30.1MB/s]


Extracting ./data/cifar-100-python.tar.gz to ./data
Files already downloaded and verified


In [5]:
# how to split test dataset?
# Split training data for validation
# Split training set into train and validation
train_size = int(0.8 * len(trainset))
val_size = len(trainset) - train_size
trainset, valset = random_split(trainset, [train_size, val_size])


In [6]:
# Create data loaders
batch_size = 32

trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)
valloader = DataLoader(valset, batch_size=batch_size, shuffle=False, num_workers=2)
testloader = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)


In [7]:
# Initialize the model, loss function, and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNNCifar().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=0.0004)

# Learning rate scheduler
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)


In [8]:
# Training function
def train(model, loader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
    return running_loss / len(loader), 100. * correct / total


In [9]:
# Evaluation function
def evaluate(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    return running_loss / len(loader), 100. * correct / total


In [10]:
# Training loop
epochs = 10
train_losses, train_accs = [], []
val_losses, val_accs = [], []
test_losses, test_accs = [], []

In [ ]:
for epoch in range(epochs):
    train_loss, train_acc = train(model, trainloader, optimizer, criterion, device)
    val_loss, val_acc = evaluate(model, valloader, criterion, device)
    test_loss, test_acc = evaluate(model, testloader, criterion, device)

    scheduler.step()

    train_losses.append(train_loss)
    train_accs.append(train_acc)
    val_losses.append(val_loss)
    val_accs.append(val_acc)
    test_losses.append(test_loss)
    test_accs.append(test_acc)

    print(f"Epoch {epoch+1}/{epochs}")
    print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")
    print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")
    print(f"Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.2f}%")
    print()

# Plot results
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Train')
plt.plot(val_losses, label='Validation')
plt.plot(test_losses, label='Test')
plt.title('Loss vs. Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(train_accs, label='Train')
plt.plot(val_accs, label='Validation')
plt.plot(test_accs, label='Test')
plt.title('Accuracy vs. Epochs')
plt.xlabel('Epochs')
plt.ylabel('Accuracy (%)')
plt.legend()

plt.tight_layout()
plt.show()

Epoch 1/150
Train Loss: 4.1178, Train Acc: 8.14%
Val Loss: 3.8531, Val Acc: 11.39%
Test Loss: 3.8015, Test Acc: 13.35%

Epoch 2/150
Train Loss: 3.7305, Train Acc: 14.21%
Val Loss: 3.6253, Val Acc: 16.08%
Test Loss: 3.5410, Test Acc: 17.42%

Epoch 3/150
Train Loss: 3.5247, Train Acc: 17.44%
Val Loss: 3.4105, Val Acc: 19.10%
Test Loss: 3.3136, Test Acc: 21.55%

Epoch 4/150
Train Loss: 3.3629, Train Acc: 20.25%
Val Loss: 3.3041, Val Acc: 21.32%
Test Loss: 3.1500, Test Acc: 25.46%

Epoch 5/150
Train Loss: 3.2454, Train Acc: 22.66%
Val Loss: 3.2512, Val Acc: 22.80%
Test Loss: 3.1149, Test Acc: 25.48%

Epoch 6/150
Train Loss: 3.1208, Train Acc: 24.76%
Val Loss: 3.0779, Val Acc: 25.57%
Test Loss: 2.8817, Test Acc: 29.65%

Epoch 7/150
Train Loss: 3.0214, Train Acc: 26.92%
Val Loss: 3.0438, Val Acc: 26.84%
Test Loss: 2.8221, Test Acc: 31.07%

Epoch 8/150
Train Loss: 2.9549, Train Acc: 28.36%
Val Loss: 2.9701, Val Acc: 28.19%
Test Loss: 2.7609, Test Acc: 32.19%

Epoch 9/150
Train Loss: 2.9011, T